# Examples of CROP
How to use CROP to remove reactions in response to a no-growth phenotype while maintaining other growth phenotypes

![Metabolic-network.jpeg](Metabolic-network.jpeg)



In [10]:
from cobra.util.array import create_stoichiometric_matrix, constraint_matrices
import cobra
import escher
import sys
import pandas as pd
import numpy as np
import cvxpy as cvx
from IPython.display import display
pd.options.display.float_format = '{:.0f}'.format
sys.path.append('../..')
from tests.test_crop import load_model


        
#model = cobra.io.load_json_model('ABC_model.json')
model = cobra.io.load_json_model('ABC_extended_model.json')
#cobra.io.save_json_model(model, 'ABC_extended.json')
stoich_matrix = create_stoichiometric_matrix(model, 'DataFrame')

def ii(name, axis=1, S = stoich_matrix):
    """Simplify the process of getting the index of the reaction."""
    if axis == 1:
        return S.columns.get_loc(name)
    else:
        return S.index.get_loc(name)
stoich_matrix

,SRC_A_e,SRC_B_e,SINK_C_e,SINK_D_e,SRC_E_e,SINK_F_e,Aim,Bim,Cex,Dex,Eim,Fex,A_to_B,A_to_C,B_to_CF,BE_to_D,C_to_F
A_ext,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0
B_ext,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0
C_ext,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
D_ext,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0
E_ext,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0
F_ext,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0
A,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0
B,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-2,-1,0
C,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,1,0,-1
D,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,2,0


In [9]:
for rxn in model.reactions:
    if 'EX_' in rxn.id:
        rxn.id = rxn.id.replace('EX_', 'SRC_')
    if 'tr' in rxn.id:
        rxn.id = rxn.id.replace('tr', 'im')
for met in model.metabolites:
    if '_e' in met.id:
        met.id = met.id.replace('_e', '_ext')
cobra.io.save_json_model(model, 'ABC_extended_model.json')

# Resolving incorrect predictions by suppressing growth 
In this section, we mathematically deal with conditions where growth is predicted, but no growth is experimentally observed. The goal here is to identify model changes that suppress *in silico* growth in a given condition. Importantly, these model changes should not have the side-effect of suppressing growth in other conditions where growth is experimentally observed, such as the condition of wild-type in minimal medium.
There are several model changes that can suppress growth. These are basically the reverse of those that restore growth. If the knocked-out gene encodes an isozyme, then removing its isozyme, or requiring both gene products as subunits of an enzyme complex, will lead to reaction removal. Reaction removal can suppress growth if the reaction is essential. Another model change that can suppress growth is to add metabolites to the biomass composition, since this increases the requirements for growth. These two model changes can also be implemented together. However, if neither of these model changes are appropriate, we can suppress growth by removing reactions from the model. To identify the reactions to remove from the model to suppress growth, we implemented an optimization. 
To illustrate the principles involved in the optimization, we first provide a generic optimization-based solution strategy. We then compare the generic strategy to the optimizations of GrowMatch [[6]](http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1000308) and Model SEED [[7](https://www.ncbi.nlm.nih.gov/pubmed/19555510),[8](http://www.nature.com/nbt/journal/v28/n9/full/nbt.1672.html)]. GrowMatch, Model SEED, and CROP all solve this problem using a large mixed integer linear program (MILP). 
All of these MILPs involve the dual linear program to FBA because  the dual characterized by the constraint that growth is not feasible. If these programs tried instead to simply set the FBA growth flux to zero, these programs would only represent that no-growth was feasible; they would not represent that growth is infeasible.

## Generic solution strategy

This strategy independently examines each condition where growth is predicted, but no growth is experimentally observed. For each such condition, a bi-level MILP is created. The purpose of this MILP is to find the optimal set of reactions to remove such that *in silico* growth becomes infeasible in this condition, but remains feasible for a set of control conditions, such as wild-type in minimal medium.
This MILP has the same variables as described previously: $S$ is the stoichiometric matrix,  $U$ is a large upper bound, $z$ is a binary decision variable that determines whether a reaction is part of the model or not, `weights` is the probability that some enzyme coded in the genome catalyzes this reaction, `minimal_growth` is a lower bound on what is considered viable growth, and `atp_maintenance` is a lower bound on how much ATP is required for viability even in the situation of no growth.

However, there are now two conditions: the condition where no growth is experimentally observed, and wild-type in minimal medium. Then, the vector $v_{no-growth}$ represents flux for the condition where no growth is experimentally observed, and has upper bounds $U_{no-growth}$; the vector $$ represents flux for wild-type in minimal medium, and has upper bounds $U_{wt}$.

$$\begin{equation}\begin{array}{ll}
\min_z  & weights^T(1-z)   \\
& 
 \begin{array}{llll}
  \max_{v_{nogrowth}}  & v_{no growth,D} \\
 & Sv_{nogrowth}=0  &  \text{ (inner problem}) \\
 &  0\leq v_{nogrowth,i} \leq U_{no growth,i}\cdot z_i  \\
\end{array} \\
& v_{nogrowth,D} \leq  \text{minimal growth}  \\
& Sv_{wt} = 0   \\
& 0 \leq v_{wt,i} \leq  U_{wt,i}\cdot z_i  \\
& v_{wt,F} \geq \text{atp maintenance} \\
& v_{wt,D} \geq \text{minimal growth} \\
& z\in \{0,1\}  \\
\end{array}\end{equation}$$

The outer minimization problem represents the modeler's optimization to find the optimal set of reactions to remove such that *in silico* growth, represented by $v_{D}$, falls below `minimal_growth`. The constraint $v_{D}\leq$ `minimal_growth` implies that when there are no solutions that reduce growth to `minimal_growth`, the MILP will not report on solutions that partially reduce growth. However, such solutions may be helpful in cases where no complete solutions can be identified. 
The inner maximization problem of the program represents FBA, although without the requirement for ATP. We removed this requirement, because it may not be satisfied in the no-growth condition.
The Program is a bi-level MILP. To transform it into a single-level MILP, the inner maximization problem must be transformed into a set of constraints. This has been done using an ingenious technique proposed by Maranas’ lab [[10]](https://www.ncbi.nlm.nih.gov/pubmed/12673766) based on the strong duality theorem of linear programming. In this technique, the inner problem is transformed into a set of constraints by including the constraints of the inner problem’s dual, and then constraining the objectives of the inner problem and the inner problem’s dual to be equal.
Before illustrating the primal and dual formulations when $z$ is included, we first show this without $z$ for simplicity. Without $z$, the primal is FBA. To represent FBA in a way that will simplify the derivation of its dual, we represent FBA without unnecessary bounds. So if there is only a single limiting nutrient, e.g. $v_{A}\leq 10$, then we represent FBA with no other upper bounds. Moreover, $v_{A}$ does not need an explicit lower bound of zero, because it will never be negative. As with the inner problem above, we also remove the ATP maintenance requirement. Then, the form of the primal LP is:

$$\begin{array}{lll}
\max_v & v_{D} \\
\text{subject to } &Sv = 0 \\
& 0\leq v_i, &\text{ for $i\neq v_A$} \\
& v_{B} \leq 0 \\
&v_{A} \leq 10
\end{array}$$

In [5]:

nmets, nrxns = stoich_matrix.shape
mets, rxns = stoich_matrix.index, stoich_matrix.columns
S = stoich_matrix.values


internal_mets = [m for m in stoich_matrix.index if '_c' in m]

c = np.zeros(nrxns)
c[ii('Dex')] = 1
v = cvx.Variable(nrxns)

primal_lp = cvx.Problem(
    cvx.Maximize( v[ii('Dex')]),
    [S@v == 0,
     0   <= v,
     0   >= v[ii('Btr')],
     10  >= v[ii('EX_A_e')]
    ]
)

primal_lp.solve(solver=cvx.SCIPY, verbose=True )
reactions = pd.DataFrame( v.value, index=rxns, columns=['v'])
display( primal_lp.constraints[1].dual_value)
display(reactions)
reactions.to_csv('primal_lp.csv')

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Oct 11 04:58:31 PM: Your problem has 17 variables, 4 constraints, and 0 parameters.
(CVXPY) Oct 11 04:58:31 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 11 04:58:31 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 11 04:58:31 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 11 04:58:31 PM: Compiling problem (target solver=SCIPY).
(CVXPY) Oct 11 04:58:31 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Con

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 4., 0., 0.])

,v
EX_A_e,10
EX_B_e,-0
SINK_C_e,-0
SINK_D_e,20
EX_E_e,10
SINK_F_e,-0
Atr,10
Btr,-0
Cex,-0
Dex,20


In [11]:
builder = escher.Builder(
    map_json='ABC_extended_map.json',
    model_json='ABC_extended_model.json',
    reaction_data= reactions
    
)
builder

Builder(reaction_data=[{'EX_A_e': 10.0, 'EX_B_e': -0.0, 'SINK_C_e': -0.0, 'SINK_D_e': 20.0, 'EX_E_e': 10.0, 'S…

Now, we need the dual. The reason we need the dual is because we want to model an upper bound on growth. To derive the dual, we follow the approach in [[11]](https://stanford.edu/~boyd/cvxbook/), which uses the  Lagrangian $L$:

$$\begin{array}{rcl}
L& = &c^Tv+ mSv - \sum_{i\neq glucose} r_i\cdot v_i+r_{glucose}\cdot(10 - v_{glucose}) \\
& = & 10r_{glucose} + (c + S^Tm - r)v \\
\end{array}$$

Then, the Lagrangian dual problem is:
$$\begin{array}{lll}
\min_{m,r}&\max_v & 10\cdot r_{glucose} + (c + S^Tm - r)v \\
& \text{subject to}& r_{glucose} \geq 0 \\
& & r_i \leq 0  \text{   for $i\neq$ glucose} \\
\end{array}$$

which is equivalent to:

$$\begin{equation}\begin{array}{lll}
\min_{m,r}& 10\cdot r_{glucose}  \\
\text{subject to}& S^Tm+c=r \\
& r_{glucose} \geq 0 \\
&r_i \leq 0 & \text{for $i\neq$ glucose} \\
\end{array}\end{equation}$$

Where $S^T$ is the transpose of the stochiometric matrix $S$, $r$ is the dual variable associated with reaction bounds (e.g. $v\geq 0$ and $v_{glucose}\leq 10$) and $m$ is the dual variable associated with the steady state constraints (that is, for all metabolites, the sum of all the fluxes that produce a metabolite is equal to the sum of the fluxes that consume a metabolite)

Now we address the case where there is a binary decision variable per reaction, $z$. To include $z$ in the optimization, we need a loose upper bound, $U$. $U$ is required to be sufficiently large such that the constraint $v_i\leq U_iz_i$ does not affect the optimum when $z_i=1$. Then, the primal becomes:

$$\begin{equation}\begin{array}{lll}
\max_v & c^Tv=v_{biomass} \\
\text{subject to} & Sv = 0 \\
& 0\leq v_i \leq U_i\cdot z_i, &\text{ for all $i$} \\
\end{array}\end{equation}$$

where $i$ indexes reactions and $U_{glucose}= 10$.  The dual to this problem is:
$$\begin{equation}\begin{array}{lll}
\min_r &\sum_i U_i\cdot z_i\cdot r_i \\
\text{subject to } &S^Tm+c = r \\
& r_i\leq\Omega_i\cdot(1-z_i) & \text{for $i\neq$ glucose} \\
& r_{glucose} \geq 0
\end{array}\end{equation}$$

where $\Omega_i$ represents a loose upper bound on the elements of the vector $r$, e.g. $\Omega_i=1000$.  This dual formulation is linear when we treat $z$ as a parameter "passed in" from the outer problem to the inner problem. However, the objective is nonlinear in the integrated single-level model, where we include  $z$ as a decision variable along with $r$.

### Resolving the nonlinearity in the general solution strategy ###
Maranas’s lab has previously shown that they can resolve the nonlinearity in program $(15)$ when $z$ is included as a decision variable [[10]](https://www.ncbi.nlm.nih.gov/pubmed/12673766). However, the solution is highly non-trivial, and we are not aware of any publication that derives it. We infer that they resolved this nonlinearity by exploiting the fact that $U_iz_ir_i=0$ for all reactions $i$ except $i=glucose$. $U_iz_ir_i=0$ for all these reactions because $U_iz_ir_i=0$ when $z_i=0$ and when $z_i=1$. If $z_i=0$, then $U_iz_ir_i=0$ trivially. If $z_i=1$ in the above problem, then the upper bound constraints on $v_i$ will not be tight, since $U_i$ is sufficiently large by design. Constraints that are not tight have dual variables of zero, so if $z_i=1$, then $r_i=0$, implying that $U_iz_ir_i=0$. The reaction $i=glucose$ is slightly different. This reaction is required, so we know it will be included in the model. Thus, we know that $z_{glucose}=1$, so $z_{glucose}$ no longer becomes a decision variable. Furthermore, because glucose is the limiting nutrient, we know that $v_{glucose}=U_{glucose}=10$, so its upper bound $U_{glucose}=10$ is tight. Tightness of this upper bound implies that $r_{glucose}\geq 0$. 
By exploiting the behavior of $U_iz_ir_i$, program (15) can be represented as, 

$$\begin{equation}\begin{array}{ll}
\min_{m,r} & 10\cdot r_{glucose} \\
\text{subject to} & S^Tm +c = r \\
& r_i\leq\Omega_i\cdot(1-z_i)\ \text{ for $i\neq$ glucose} \\
& r_{glucose} \geq 0 \\
\end{array}\end{equation}$$

Program $(16)$ does not possess the nonlinearity of program $(15)$. Thus program $(16)$ can be used to transform program $(12)$ into a single-level MILP by transforming the inner problem into a set of constraints.

$$\begin{equation}\begin{array}{l}
\min_z weights^T(1-z) \\
\begin{array}{lll}
& v_{biomass} = 10r_{glucose} \\
& Sv= 0 & \text{inner problem} \\
& 0\leq v_i\leq U_{no growth,i}\cdot z_i \\
& S^Tm +c = r \\
& r_i\leq\Omega_i\cdot(1-z_i) & \text{for $i\neq$ glucose} \\
& r_{glucose} \geq 0 \\
\end{array}\\
v_{biomass} \leq\text{minimal growth} \\
Sw= 0 \\
0\leq w_i\leq U_{wt,i}\cdot z_i \\
w_{biomass} \geq \text{minimal growth} \\
w_{ATP} \geq \text{atp maintenance} \
z\in \{0,1\} \\
\end{array}\end{equation}$$


### Size 

FBA maximizes growth, so its dual represents an upper bound on growth. Thus, to mathematically express the experimental observation of no-growth (or of growth below a threshold), we need the dual. However, it also follows that once we have the dual, we no longer need FBA. In fact, incorporating both FBA and its dual only provides value if we want to model an intermediate level of growth. This observation appears to have been overlooked. However, only using the dual substantially reduces the MILP’s size.
This observation is actually suggested by program  (17) in *Generic solution strategy*. Program  (17) includes the constraints $v_{biomass}=10r_{glucose}$ and it includes $v_{biomass}\leq$ `minimal_growth`. Together, these constraints imply that $10r_{glucose}\leq$ `minimal_growth`, and that we can ignore $v_{biomass}$. When we remove FBA’s growth maximization from the inner problem of program (17), we get:

$$\begin{array}{ll}
\min_z &  weights^T(1-z) \\
 &\begin{array}{lll}
& S^Tm -r = -c \\
& r_i\leq\Omega_i\cdot(1-z_i) & \text{for $i\neq$ glucose} \\
& r_{glucose} \geq 0 \end{array} \\
& 10r_{glucose} \leq \text{minimal growth} \\
& Sw = 0 \\
& 0\leq w_i \leq U_{wt,i}\cdot z_i \\
& w_{biomass} \geq \text{minimal growth} \\
& w_{ATP} \geq \text{atp maintenance} \\
& z\in \{0,1\}
\end{array}$$

### Relaxing the minimal growth solution

One issue with the Generic solution strategy is that when there are no solutions that reduce growth to `minimal_growth`, solutions that partially reduce growth are not reported. To overcome this issue, $r_{glucose}$ can be moved to the objective.

$$\begin{array}{ll}
\min_z &  weights^T(1 - z)  + M\cdot r_{B}\\
 &\begin{array}{lll}
& S^Tm -r = -c \\
& r_i\leq\Omega_i\cdot(1-z_i) & \text{for $i\neq B$} \\
& r_{B} \geq 0 \end{array} \\
& Sw = 0 \\
& 0\leq w_i \leq U_{wt,i}\cdot z_i \\
& w_{biomass} \geq \text{minimal growth} \\
& w_{ATP} \geq \text{atp maintenance} \\
& z\in \{0,1\}
\end{array}$$

where the constant $M$ balances
1. the model's consistency with reaction weights versus
2. consistency with growth phenotypes


One issue with the Generic solution strategy is that when there are no solutions that reduce growth to `minimal_growth`, solutions that partially reduce growth are not reported. To overcome this issue, $r_{B}$ can be moved to the objective.

$$\begin{array}{ll}
\min_z &  weights^T(1 - z)  + M\cdot r_{EX_B}\\
 &\begin{array}{lll}
& S^Tm -r = -c \\
& r_i\leq\Omega_i\cdot(1-z_i) \text{ for $i\neq EX_B$} \\
& r_{EX_B} \geq 0 \text{  $B$ is limiting}\\
& r_{EX_A} \geq 0  \text{ $A$ is limiting}\end{array} \\
& Sw = 0 \\
& 0\leq w_A \leq U_{wt,A} \\
& z_A = 1 \\
& 0\leq w_i \leq U_{wt,i}\cdot z_i \\
& w_{D} \geq \text{minimal growth} \\
& w_{F} \geq \text{atp maintenance} \\
& z\in \{0,1\}
\end{array}$$

where the constant $M$ balances
1. the model's consistency with reaction weights versus
2. consistency with growth phenotypes

## Defining the phenotypes that exclude EX_B

If we want to create a phenotype that implies we should remove the `EX_B: B_E => B` transport reaction, then we need a phenotype that says, we can grow on `A_e` as a nutrient source, but we cannot grow on `B_e` as a nutrient source.


In [21]:

nmets, nrxns = stoich_matrix.shape
mets, rxns = stoich_matrix.index, stoich_matrix.columns
S = stoich_matrix.values
minimal_growth  = 0.1
atp_maintenance = 0.1
def idx(name, axis=1, S = stoich_matrix):
    if axis == 1:
        return S.columns.get_loc(name)
    else:
        return S.index.get_loc(name)
internal_mets = [m for m in stoich_matrix.index if '_c' in m]
B       = 10
U       = np.eye(nrxns)*10.0
weights = np.ones(nrxns)*0.5
Omega   = 1000.0
c = np.zeros(nrxns)
c[idx('SINK_D')] = 1
w = cvx.Variable(nrxns)
m = cvx.Variable(nmets)
r = cvx.Variable(nrxns)
z = cvx.Variable(nrxns, boolean=True)

dung_fba_partially_reduced = cvx.Problem(
    cvx.Minimize( weights.T@(1-z) + B*r[idx('EX_B_e')]),
        [       S.T@m - r       == -c,
                r[:idx('EX_B_e')]     <= Omega*np.eye(idx('EX_B_e'))@(1 - z[:idx('EX_B_e')]),
                r[idx('EX_B_e')]      >= 0,
                r[idx('EX_B_e')+1:]   <= Omega*np.eye(nrxns - idx('EX_B_e') - 1)@(1 - z[idx('EX_B_e') + 1:]),
            S@w             == 0,
            0               <= w,
            w               <= U@z,
            w[idx('SINK_D')]      >= minimal_growth,
            w[idx('SINK_F')]          >= atp_maintenance])
dung_fba_partially_reduced.solve(solver=cvx.SCIPY, )
reactions = pd.DataFrame( z.value, index=rxns, columns=['z'])
reactions['r'] = pd.Series(np.squeeze(np.asarray(r.value)),index=rxns)
reactions['w'] = pd.Series(np.squeeze(np.asarray(w.value)),index=rxns)
display(reactions)
metabolites = pd.DataFrame(m.value,index=mets, columns=['m'])
display(metabolites)

,z,r,w
EX_A_e,1,-1,0
EX_C_e,1,-1,0
EX_D_e,1,0,0
EX_E_e,1,-2,0
EX_F_e,1,0,0
EX_A,1,-0,-0
EX_C,1,-0,-0
SINK_D,1,0,0
EX_E,1,-0,0
SINK_F,1,0,0


,m
A_c,1
B_c,0
C_c,1
D_c,1
E_c,2
F_c,0
A_e,1
C_e,1
D_e,-0
E_e,2
